# FIT5196 Assessment 2
#### Student Name: Jianhan Ma & Yi Hsiao
#### Student ID: 29332885 & 29360595
#### Group ID: 023
Date: 10/09/2019

Version: 2.0

Environment: Python 3.6.0 and Anaconda 4.3.0 (64-bit)

Libraries used:
* pdfminer (for parsing PDF file, included in Anaconda Python 3.6)
* pandas 0.19.2 (for data frame, included in Anaconda Python 3.6) 
* re 2.2.1 (for regular expression, included in Anaconda Python 3.6) 
* nltk 3.2.2 (Natural Language Toolkit, included in Anaconda Python 3.6)
* nltk.collocations (for finding bigrams, included in Anaconda Python 3.6)
* nltk.tokenize (for tokenization, included in Anaconda Python 3.6)
* nltk.corpus (for stop words, not included in Anaconda)
* nltk.stem.porter(for extraction of stem)

## 1. Introduction

This task provide a data-set containing 200 URLs for papers published in a popular AI conference. extracting data from nonstructured format and converting the extracted data into a proper format suitable for a downstream modelling task is the main goal of this assignemnt.

More details for each task will be given in the following sections.

## 2.  Import libraries 

In [3]:
import requests
import re
import os
import pdfminer
import nltk
from nltk.tokenize import RegexpTokenizer 
from nltk.tokenize import MWETokenizer
from nltk.stem.porter import PorterStemmer
from nltk.corpus import gutenberg
from nltk.tokenize.punkt import PunktSentenceTokenizer, PunktTrainer
from nltk.collocations import *
import collections
from collections import *
import pandas as pd
from itertools import chain
import itertools
nltk.download('gutenberg') ##sentence segmentation

[nltk_data] Downloading package gutenberg to
[nltk_data]     C:\Users\xxxx\AppData\Roaming\nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


True

## 3. Extract pdf file and get the URLs

As a first step, the file `Group023.pdf` will be converted to txt file so its can be readable.

In [2]:
##convert pdf to txt
!pdf2txt.py -o Group023.txt Group023.pdf

Active code page: 65001


## 4. Sparse Feature Generation

### 4.1 Download all paper and transfer them to txt file

After getting the txt file, the `paper ID` and `URL` should be extracted

In [4]:
##read stop word file
pdfTxtFile = 'Group023.txt'
with open(pdfTxtFile, 'r',encoding='UTF-8') as pdf_txt:
    inputfile = []
    useless = 'filename|url|\s' ##remove irrevelant elements
    # loop over all the lines
    for line in pdf_txt.readlines():
        line = line.strip('\n')
        line = re.sub(useless,'',line)
        inputfile.append(line)
    while '' in inputfile:
        inputfile.remove('') ##remove blank elements
inputfile[0]

'PP3164.pdfhttps://drive.google.com/uc?export=download&id=1KGospAeT509Iy4az0wnZc1aNNEF44Lfb'

In the next three steps, we use three similar but different groups to extract url, pdf name ,and txt name(txt name is for following steps)

In [5]:
##get the url from list
def get_url(url):
    url_pattern = 'PP.*pdf(https.*)'
    url = re.match(url_pattern,url)
    if url != None:
        url = url.group(1)
        return url
get_url(inputfile[0])

'https://drive.google.com/uc?export=download&id=1KGospAeT509Iy4az0wnZc1aNNEF44Lfb'

In [6]:
def get_pdf_name(url):
    name_pattern = '(PP.*pdf)https.*'
    name = re.match(name_pattern,url)
    if name != None:
        name = name.group(1)
        return name
get_pdf_name(inputfile[0])

'PP3164.pdf'

In [7]:
def get_txt_name(url):
    txt_pattern = '(PP.*).pdfhttps.*'
    name = re.match(txt_pattern,url)
    if name != None:
        name = name.group(1)
        return name +'.txt'
get_txt_name(inputfile[3])

'PP3249.txt'

Then we define `download` function by using `requests.get()` method and the input parameter is url we got from previous step. And the for loop is to call the `download` function to download all the paper as pdf version.(it may spend a quite long time)

In [1]:
def download(filename, url):
    res = requests.get(url)##use request package to read url
    fp  = open(filename, "wb")
    content = res.content ##While .content can access to the raw bytes of the response payload
    fp.write(content)
    fp.close()

In [80]:
##download all the pdf file
for i in range(len(inputfile)):
    filename = get_pdf_name(inputfile[i])
    url = get_url(inputfile[i])
    download(filename, url)

'\nfor i in range(len(inputfile)):\n    filename = get_pdf_name(inputfile[i])\n    url = get_url(inputfile[i])\n    download(filename, url)\n'

This part is for converting all paper to txt file. `$` symbol is to call the value which is in the variable.

In [81]:
##convert all pdf file to txt
for i in range(len(inputfile)):    
    pdfname = get_pdf_name(inputfile[i])
    txtname = get_txt_name(inputfile[i])
    !pdf2txt.py -o $txtname $pdfname ##use '$' to get the value(shell script)

'\nfor i in range(len(inputfile)):    \n    pdfname = get_pdf_name(inputfile[i])\n    txtname = get_txt_name(inputfile[i])\n    !pdf2txt.py -o $txtname $pdfname\n'

### 4.2 Get paper body and tokenization

For this part, we use `re` package to extract body from a paper and replace all strings which may have bad effect following parts.

In [8]:
def get_body(url):
    txt_name = get_txt_name(url)
    with open(txt_name,'r',encoding='UTF-8') as content:
        text = content.read()
    
    text = text.replace('\n',' ')
    text = text.replace('?','')
    text = text.replace('¿','')
    text = text.replace('ﬃ','ffi')
    text = text.replace('ﬂ','fl')
    text = text.replace('ﬀ','ff')
    text = text.replace('ﬁ','fi')
    
    body_pattern = '1\sPaper\sBody(.*)2\sReferences'
    body = re.search(body_pattern,text)
    body = body.group(1)
    
    return body
get_body(inputfile[1])[:200]

'  Current reinforcement-learning (RL) techniques hold great promise for creating a general type of artificial intelligence (AI), specifically autonomous (software) agents that learn difficult tasks wi'

For sentence segmentation, I read [PunktSentenceTokenizer](https://nlpforhackers.io/splitting-text-into-sentences/) as a reference to train a Punkt Sentence Tokenizer. It will be useful in `tokenization` function.

In [9]:
## sentence segnmentation
text = ""
for file_id in gutenberg.fileids():
    text += gutenberg.raw(file_id)
    
trainer = PunktTrainer()
trainer.INCLUDE_ALL_COLLOCS = True ##wether contain collocations or not
trainer.train(text)

Read `stopwords.txt` file and store in a set.

In [10]:
##read stop word file
with open('stopwords_en.txt') as stopwords:
    stopwords = stopwords.read()
stopwords_list = re.split('\n', stopwords)
stopwords_set = set(stopwords_list)

Then in order to tokenize the words in the body, the paper body should be integrated into a dictionary(key is paper ID, value is body).

In [11]:
##create dictionary to store all bodies
body_raw = {}
all_pid = []
for i in range(len(inputfile)): 
    all_body = "".join(get_body(inputfile[i])) ## get all body by using 'get_body' function.
    pid = get_pdf_name(inputfile[i])
    pid = pid.replace('.pdf','')
    body_raw[pid] = all_body ## store paper id and its body into a dictionary
    all_pid.append(pid)

define a `RegexTokenizer` to tokenize all words from paper.

In [12]:
tokenizer = RegexpTokenizer(r"[A-Za-z]\w+(?:[-'?]\w+)?")

Utilize the trainning we defined in last few parts to lowercase normal characters. And then tokenize all the tokens from body.

In [13]:
def tokenizebody(pid):
    ##normalized to lowercase except the capital tokens appearing in the middle of a sentence/line.
    lower_tokenizer = PunktSentenceTokenizer(trainer.get_params())
    raw_body = body_raw[pid]
    sentences = lower_tokenizer.tokenize(raw_body)
    sentences_lower = [s[0].lower()+s[1:] for s in sentences]
    raw_body = "".join(sentences_lower)
    ## tokenize words
    tokenized_body = tokenizer.tokenize(raw_body)
    return (pid, tokenized_body) # return a tupel of patent_id and a list of tokens



In [14]:
##get all tokens
tokenized_body = dict(tokenizebody(pid) for pid in body_raw.keys())
tokenized_body['PP7260'][0:10]

['Neuropsychiatric',
 'disorders',
 'schizophrenia',
 'autism',
 'spectral',
 'disorder',
 'etc',
 'take',
 'an',
 'enormous']

In order to get all words from dictionary, The [itertools tutorila](https://docs.python.org/2/library/itertools.html) is really helpful to sepreate words and integrate them.

In [18]:
all_words = list(chain.from_iterable(tokenized_body.values()))
len(all_words)

806122

The context-independent stop words should be removed from the vocabulary(using `stopwords_set` we defined before).

In [15]:
##remove all stop_word
stopped_tokens = [w for w in all_words if w not in stopwords_set]## append words from tokens who is not stop word
len(stopped_tokens)

439378

For generating 200 meaningful collocations task. The functions we need include
* BigramAssocMeasures()
* BigramCollocationFinder.from_words()
* apply_freq_filter(20)
* apply_word_filter(lambda w: len(w) < 3)
* nbest(bigram_measures.pmi, 200)

which are come from http://www.nltk.org/howto/collocations.html. 

In [16]:
## generate 200 meaningful collocation
bigram_measures = nltk.collocations.BigramAssocMeasures()
bigram_finder = nltk.collocations.BigramCollocationFinder.from_words(stopped_tokens)
bigram_finder.apply_freq_filter(20)
bigram_finder.apply_word_filter(lambda w: len(w) < 3)# or w.lower() in ignored_words)
top_200_bigrams = bigram_finder.nbest(bigram_measures.pmi, 200) # Top-200 bigrams
len(top_200_bigrams)

200

Now, we introduce 200 collcations. we need to make sure those collocations are not split into two individual words.

In [17]:
mwetokenizer = MWETokenizer(top_200_bigrams)
colloc_paper =  dict((pid, mwetokenizer.tokenize(body)) for pid,body in tokenized_body.items())
all_words_colloc = list(chain.from_iterable(colloc_paper.values()))
colloc_voc = list(set(all_words_colloc))

check the difference between th output of MWEtokenizer and RegexpTokenizer by adpating the following code to make sure we get the right collocation

In [18]:
for pid in tokenized_body.keys():
    diff = set(colloc_paper[pid])-set(tokenized_body[pid])
    if len(diff) != 0:
        print (pid, diff)

PP3164 {'mutual_information', 'junction_tree', 'oracle_calls', 'special_case', 'dynamic_programming', 'fig_shows', 'closely_related', 'upper_bound', 'upper_bounds', 'global_optimum'}
PP3197 {'reinforcement_learning', 'decision_making', 'Related_Work', 'computational_complexity', 'lower_bound', 'upper_bound'}
PP3199 {'standard_deviation', 'reinforcement_learning', 'Related_Work', 'figure_shows', 'storm_cell', 'perfor_mance'}
PP3249 {'worst_case', 'prob_lem', 'special_case', 'lower_bound', 'upper_bound'}
PP3279 {'free_energy', 'Markov_Random', 'lateral_connections', 'square_root', 'visible_units', 'figure_shows', 'deep_belief', 'weight_decay', 'lower_bound', 'hidden_units'}
PP3282 {'Monte_Carlo', 'standard_deviation', 'marginal_likelihood', 'free_energy', 'acknowledgments_This', 'dynamical_systems', 'figure_shows', 'hand_side', 'algo_rithm', 'upper_bound', 'computationally_efficient', 'gradient_descent', 'error_bars', 'maximum_likelihood'}
PP3320 {'randomly_selected', 'random_walk', 'fig

Rare tokens (with the threshold set to 3%) and meaningless tokens (with the threshold set to 95%) must be removed from the vocab. The iteration that I write will spend a quite long time. So I store the result into a txt file for the moment to save time.  

In [25]:
##save Rare tokens (with the threshold set to 3%) and context-dependent (with the threshold set to %95)
words = []
all_words_set = list(set(all_words))
count = 0 ##give a 0 value to count
for x in all_words_set:
    for i in all_pid:
        this_body_words = list(set(tokenized_body[i])) ## to get tokends of current paper
        ## if find this word in a paper, count + 1
        if x in this_body_words:
            count += 1
            
        ## if result is in this range, record this word.
    print(x + ':' + str(count/200))## see document frequency for each token
    if count/len(all_pid) >= 0.95 or count/len(all_pid) < 0.03:
        words.append(x)
    count = 0 ##initialize the counter for each loop
words = list(set(words))

suggestions:0.03
de2:0.005
videos4:0.005
parameterselection:0.005
restraint:0.005
aj:0.065
teacher:0.025
processor:0.025
max-normalized:0.005
nR:0.01
subcellular:0.005
Recently:0.03
taminated:0.005
ample:0.025
Vars:0.005
surprised:0.005
px1:0.005
Takeda:0.005
faithfulness:0.01
SSC:0.01
Gilbert:0.005
genetically-encoded:0.005
dictive:0.005
partment:0.005
Globerson:0.005
betas:0.005
SurK:0.005
refseq:0.005
physically:0.025
Long-Term:0.005
market:0.04
native:0.015
tracking:0.065
tij:0.01
concentrates:0.005
dJ:0.005
estimatorTV:0.005
pan:0.01
alternatingly:0.015
sachusetts:0.01
conditions:0.435
trip:0.01
broadcasted:0.005
EfL:0.005
guages:0.005
level-set:0.005
Cn:0.025
ec:0.01
experimentation:0.01
slowly:0.08
seeking:0.03
elim:0.005
convex-concave:0.015
Numerical:0.055
jye02:0.005
metal:0.005
IK:0.005
meantime:0.01
did:0.3
Model-4:0.005
P2n500r25:0.005
crawled:0.005
pre-processed:0.015
convolumatching:0.005
Near-optimal:0.005
label-wise:0.005
abound:0.01
dvdelectronics:0.005
non-iconic:0.0

trivially:0.075
KW14:0.005
Invp:0.005
formulation:0.33
CCF:0.005
shtml:0.005
pre-compute:0.01
Charitable:0.015
feedbacks:0.015
pathwise:0.005
Ldata:0.005
encode:0.14
Anew:0.005
dx:0.095
bengio:0.005
flavour:0.005
bl0:0.005
multi-type:0.005
assume:0.77
computationally-efficient:0.005
phe:0.01
anatomical:0.015
acteristics:0.01
off-line:0.005
sandwich:0.005
between-lobe:0.005
texts:0.015
cancellation:0.005
GEEM:0.005
Non-stick:0.005
endorsements:0.01
goto:0.005
please:0.07
subroutines:0.005
P3n500r25:0.005
asymmetries:0.005
proximity-bias:0.005
D9:0.005
art:0.18
Nove:0.005
Null:0.005
ani:0.005
Schedule:0.005
EXPX:0.005
Moment:0.005
ZX12:0.005
undersample:0.005
departs:0.01
Correspondences:0.005
specialist:0.01
YO:0.005
avatar:0.005
iRk:0.005
chemistry:0.01
rounding:0.02
Advantage:0.005
determinants:0.005
visualization:0.07
eTj:0.005
continuity:0.035
rL:0.015
Ros:0.005
Ballin-a:0.005
effe:0.005
Jascha:0.005
Lc:0.005
XY:0.025
capped:0.01
treatment-control:0.005
Carota:0.005
frequentist:0.00

FKM05:0.005
sklearn:0.005
finished:0.01
technology:0.04
ulus:0.01
pedestrian:0.01
corners:0.025
cavity:0.01
tk1:0.01
af:0.02
CanICA:0.005
geographic:0.005
one-half:0.005
power-method:0.005
AUPRG:0.005
describ:0.005
upfront:0.005
minv:0.01
converted:0.06
yellow:0.035
minkx:0.005
Violated:0.01
coefficient:0.11
MR8:0.005
maxkx:0.005
carmounted:0.005
zspec:0.005
Dv:0.01
Promotion:0.005
desired:0.225
temporal-occipital:0.005
wake:0.01
autoenInitialize:0.005
Amodei:0.005
ciently:0.025
kindly:0.01
non-submodularity:0.005
sprial:0.005
prop:0.065
referenced:0.015
slices:0.01
Alfred:0.01
causes:0.115
rod:0.005
posed:0.105
warping:0.01
qT0:0.005
mil:0.01
Optimize:0.005
kNh:0.005
tran:0.005
eg:0.025
monetary:0.005
zx:0.005
non-Euclidean:0.005
sonar:0.005
Samsung:0.02
was:0.9
pretrained:0.02
feasible:0.165
different:0.92
initializer:0.005
profile:0.045
neighboring:0.1
ill-conditioned:0.01
Python:0.035
regularizer2:0.005
blocked:0.025
dogma:0.005
ErD:0.005
song:0.005
contemporaneous:0.005
Eqn:0.02
l

Eigendecomposition:0.005
GLS81:0.005
d2r:0.005
filter:0.115
serves:0.08
Yil:0.005
gripper:0.005
HM:0.01
kKt:0.005
volunteers:0.005
ACE:0.005
cosets:0.005
subpolicies:0.005
admissions:0.005
composable:0.01
L2LSTD:0.005
Nearset:0.005
double-or:0.005
tempt:0.01
Wx0j:0.005
DetectorNet1:0.005
ParaHippocampal:0.005
Subgroups:0.005
difficultly:0.005
joint-actions:0.005
docenti:0.005
multiplicaT:0.005
testify:0.005
mid-1950s:0.005
Zl1:0.005
rated:0.02
LSB:0.005
CASA:0.005
allocation:0.035
sss1:0.005
Dj:0.005
navigational:0.005
ning:0.03
sin:0.09
Berkes:0.005
Apprenticeship:0.005
Egregious:0.005
itpwere:0.005
xr:0.02
word-alignment:0.005
genis:0.005
eliciting:0.01
avg:0.015
cave:0.005
enormously:0.005
Hart:0.005
funtion:0.005
appeals:0.01
rons:0.015
hallmark:0.01
simply:0.56
hierarchically:0.01
MIT:0.01
cre:0.005
paremeters:0.005
Boltzmann:0.05
sometime:0.005
band:0.04
handbag:0.005
SG:0.01
architectures:0.11
reaction:0.01
mistakes:0.05
FP7-ICT:0.005
flow-based:0.005
extrapolates:0.01
hetereoge

AGKM12:0.005
Lyu:0.01
IIS-1117591:0.005
FAlap:0.005
opportunities:0.03
RCV1:0.005
topic-weight:0.005
claim:0.1
assigns:0.09
concatena:0.005
experimentally:0.105
exp-concave:0.005
kP:0.025
dynamical:0.065
matics:0.005
ainv:0.005
Dec-POMDPs:0.005
spawning:0.005
benchmarks:0.055
Sigmoid:0.01
fG:0.005
Alekh:0.005
S2Z:0.005
Frequencies:0.005
ga0:0.005
amazondslr:0.005
presser:0.005
faithful:0.02
Cutajar:0.005
relief:0.005
gen:0.12
IQP:0.005
proportionality:0.005
Panel:0.015
ERG:0.005
Ilan:0.005
softmax:0.06
truG-TRBM:0.005
TruG-RBM:0.005
TWV:0.005
Id:0.02
utmost:0.005
chi-squared:0.01
dilation:0.005
Ei:0.03
LWTA-n:0.005
Seventh:0.025
CPRL:0.005
tautology:0.005
xzi:0.005
Bengio:0.035
degree:0.24
compared:0.66
Ratio:0.015
copiesto:0.005
submits:0.005
SimpleIS:0.005
resolution:0.095
increas:0.015
Lamb:0.005
freely:0.02
HN-ST:0.005
zij:0.015
boosted:0.025
uiSk:0.005
Hubs:0.005
mMD-quad:0.005
angularly:0.005
Auto-encoders:0.005
supple:0.01
PC14:0.005
appends:0.005
visit:0.03
cQ:0.005
peer:0.005


...

continuation:0.02
pre-process:0.015
d1j:0.005
serial:0.005
pl:0.005
Spiking:0.005
ronment:0.005
DRV10:0.005
ill-posed:0.015
Decoupling:0.005
MovieLens-1Million:0.005
lower-triangular:0.005
grants:0.065
everywhere:0.05
catch-up:0.005
smoothers:0.005
third-party:0.005
condence:0.005
Bowman:0.005
evance:0.01
OT:0.01
exemplify:0.005
zi:0.11
promoters:0.005
N000141310817:0.005
Coates:0.005
CORD-0008:0.005
Lik:0.005
dropping:0.045
pool7:0.005
bedding:0.03
P17229-N04:0.005
scan:0.025
chang:0.01
decentralized:0.01
Composites:0.005
multi-lingual:0.005
difference:0.545
ASAP:0.005
N1T:0.005
Send:0.005
fact:0.66
cC0:0.005
fulfilled:0.03
anirudh9119:0.005
Raytheon:0.005
mX0:0.005
all-threshold:0.005
DavisKahan:0.005
ss-SRTRBM:0.005
Fynn:0.005
obsolete:0.005
bnjk:0.005
A4n:0.005
Bq:0.005
tackle:0.08
Markov:0.26
parade:0.005
No-regret:0.005
logQ:0.005
Dudik:0.005
Four:0.01
losing:0.01
expire:0.005
Extraction:0.02
play:0.145
conclu:0.005
Ak1:0.005
Zj:0.01
above:0.83
Reward-Weighted:0.005
devel:0.005
h

tual:0.005
right-hand:0.07
connected:0.28
inter-dependencies:0.005
prob:0.24
Occurrence:0.01
evolution:0.085
LIBSVM:0.02
error-types:0.005
tune:0.1
logtrans:0.005
co-authors:0.01
ceptive:0.005
suffering:0.005
Privacy:0.005
Evidence:0.005
unif:0.005
Propo:0.005
sadness:0.005
convolu:0.015
Announce:0.005
rule7:0.005
Stream:0.005
estiAccept:0.005
authorship:0.005
Pennington:0.005
simplicty:0.005
captioning:0.005
amazon:0.005
Quadratic:0.015
z2:0.03
Yn:0.02
instabilities:0.025
aggregator:0.005
query-document:0.005
dRd:0.005
quantitative:0.115
probes:0.01
sn1:0.005
bookstein:0.005
includedcan:0.005
Spectral:0.025
DBN3Ramp:0.005
Wxj:0.005
Convolutional:0.04
post-transcritional:0.005
notable:0.09
paradigm:0.06
asked:0.035
centrally:0.005
Jaakkola:0.01
cur:0.03
gik:0.005
Yvc:0.005
eC:0.01
retrieval:0.1
planar:0.015
fml:0.005
strength:0.11
QLDS:0.005
generalizability:0.01
discretized:0.09
Vilnis:0.005
tumor:0.01
mex:0.005
remarks:0.045
computed:0.58
Walker:0.005
Chernoff-Stein:0.005
batting:0.0

submodular:0.06
iro:0.005
C0s:0.005
overlapped:0.01
Litke:0.005
CE-LWR:0.005
i7-3840QM:0.005
H2:0.03
expressiveness:0.015
A7:0.005
qk2H:0.005
ADAM:0.005
Shannon:0.015
asymptot:0.005
Foundation:0.16
kP1:0.005
unthresholded:0.005
loans:0.005
suboptimality:0.03
maxst:0.005
Action:0.025
deliver:0.02
vacation:0.01
empirics:0.005
arrives:0.035
lymphocytes:0.005
Topic-5:0.005
Ft:0.025
timeto-success:0.005
Do:0.02
Initially:0.005
strap:0.005
boundedmemory:0.005
caregivers:0.005
ranked:0.06
med2:0.005
York:0.01
re-training:0.01
nota:0.005
fidelity:0.03
boxed:0.005
USERS:0.005
RMS:0.015
VC-bound:0.005
LTCI:0.005
pr0:0.005
Differentiation:0.005
Variance:0.02
R01GM103544:0.005
patches:0.075
Coupe:0.005
spondences:0.005
s0i:0.01
samlping:0.005
induc:0.015
PropositionalPred:0.005
k2f0:0.005
Uk2:0.005
chains:0.055
antees:0.025
meet:0.035
Cyy2:0.005
Schematic:0.005
unit-length:0.005
G22:0.005
Ci:0.06
cODE:0.005
collaborations:0.005
pairing:0.005
sVRG-ADMM:0.005
wA:0.005
tors:0.06
Amari:0.005
Approach:

interleavings:0.005
teractions:0.02
Joachims:0.005
regions:0.22
terpipe:0.005
Barcelona:0.165
Examination:0.005
self-modularize:0.005
Associating:0.005
blogging:0.005
pivotal:0.015
tth:0.02
Discovery:0.02
NUTS:0.005
Herzliya:0.005
Electric:0.01
MNIST:0.14
lagged:0.01
worthwhile:0.02
popula:0.005
lattice:0.025
Critically:0.005
appears:0.23
ni:0.115
legends:0.005
Initialization:0.03
Witwicki:0.005
procedural:0.005
typifies:0.005
PnP:0.005
lutive:0.005
mD:0.005
sE-MPP:0.005
representativeness:0.005
top-to:0.005
initiate:0.01
criterion:0.205
Ncuts:0.005
kernelized:0.02
adaptively:0.07
Parrotate:0.005
XinHua:0.005
prj:0.005
signalin:0.005
tiMINo-linear:0.005
maxAV:0.005
energy-consumption:0.005
any:0.925
deletion:0.015
Detect:0.005
LiveJournal:0.005
adjusted:0.03
iological:0.005
Hakimi:0.005
integrated:0.07
mini-batch:0.075
Re-compute:0.005
re-parameterized:0.01
Cii:0.005
CDL:0.005
positing:0.01
scarce:0.02
cubically:0.005
clinical:0.035
exemplifies:0.005
PC15:0.005
Berlin:0.01
heights:0.00

fpr:0.005
sliding-window:0.005
disagreeing:0.005
culates:0.005
mine:0.03
hyperparmeters:0.005
https:0.115
SONGS:0.005
stimuli:0.035
elicitation:0.005
psychoacoustics:0.005
recapitulate:0.005
xti:0.01
resistors:0.005
factorizations:0.01
data-submanifold:0.005
meanwhile:0.06
rewrite:0.085
academic:0.01
List:0.01
clasof:0.005
c4:0.025
mensions:0.01
protsubloc:0.01
all-ones:0.005
Karpathy:0.005
provided:0.565
Refer:0.005
Op:0.02
sub-location:0.005
know:0.295
documents:0.055
shogun-toolbox:0.005
kPd:0.005
ily:0.015
infers:0.025
intrigued:0.005
accumulators:0.005
RichardsonRomberg:0.005
adopters:0.01
nc2:0.01
undecided:0.005
sphere-constrained:0.005
yCn:0.005
tagsets:0.005
periodic:0.02
LL:0.025
kgkL2:0.005
neg:0.015
Technologies:0.005
accrue:0.005
putation:0.015
GBA:0.005
Emission:0.005
forgetting:0.03
context-free:0.005
verses:0.005
OCO:0.01
Aenon:0.005
FESS-MF:0.005
FDG:0.005
pyrimidines:0.005
geograph:0.005
Bayes-Bayes:0.005
denoiser:0.005
re-parameterization:0.005
Muirhead:0.005
every:0

The next two part is not necessary, just for saving time when I restart this code.

In [26]:
##save Rare tokens (with the threshold set to 3%) and context-dependent (with the threshold set to %95)
with open('unsatisfactory_words.txt','w+',encoding='UTF-8') as unsatisfactory_words:
    for i in words:
        unsatisfactory_words.write(i+'\n')

In [27]:
##read stop word file
with open('unsatisfactory_words.txt') as unsatisfactory_words:
    unsatisfactory_words = unsatisfactory_words.read()
unsatisfactory_words_list = re.split('\n', unsatisfactory_words)

clean_tokens = [w for w in colloc_voc if w not in unsatisfactory_words_list]## same as remove stopwords part
clean_tokens.sort()

Unigram tokens should be stemmed using the Porter stemmer. [PorterStemmer](https://pythonprogramming.net/stemming-nltk-tutorial/) is useful to acheive this task.

In [28]:
##Store all the token which contains upper case(because PorterStemmer performs lower casing by default)
upper_token = []
for word in clean_tokens:
    for i in range(len(word)):
        if word[i].isupper() == True:##use .isupper() function to get the upper case
            upper_token.append(word)

##Porter stemmer           
porter_stemmer = PorterStemmer()
stemmed_token = []
for i in range(len(clean_tokens)):
    stemmed_token.append(porter_stemmer.stem(clean_tokens[i]))

stemmed_token.extend(upper_token)##get back upper case.
stemmed_token.sort() ##sort in alphabetical ascending order.
stemmed_token

['A0',
 'A0',
 'A1',
 'A1',
 'A2',
 'A2',
 'A3',
 'A3',
 'ADMM',
 'ADMM',
 'ADMM',
 'ADMM',
 'ANR',
 'ANR',
 'ANR',
 'AP',
 'AP',
 'AP',
 'AT',
 'AT',
 'AT',
 'AUC',
 'AUC',
 'AUC',
 'Accuracy',
 'Acknowledgements',
 'Acknowledgments',
 'Ada_Newton',
 'Ada_Newton',
 'Adam',
 'Adaptive',
 'Additionally',
 'Advanced',
 'Adversarial',
 'After',
 'Agency',
 'Ai',
 'Ai',
 'Air',
 'Ak',
 'Ak',
 'Alg',
 'Algorithm',
 'Algorithms',
 'All',
 'Also',
 'Although',
 'Amazon',
 'An',
 'An',
 'Analysis',
 'And',
 'Another',
 'Any',
 'Ap',
 'Ap',
 'Appendix',
 'Application',
 'Applications',
 'Apply',
 'Approach',
 'Approximate',
 'Approximation',
 'Architecture',
 'Arora',
 'As',
 'As',
 'Assume',
 'Assumption',
 'Assumptions',
 'At',
 'At',
 'Average',
 'Avg',
 'Award',
 'Ax',
 'Ax',
 'B1',
 'B1',
 'BFGS',
 'BFGS',
 'BFGS',
 'BFGS',
 'Bach',
 'Background',
 'Based',
 'Basic',
 'Bayes',
 'Bayes_classifier',
 'Bayesian',
 'Beach',
 'Because',
 'Before',
 'Bellman',
 'Bellman_error',
 'Bengio',
 'Bern

Remove tokens with the length less than 3 from the vocab

In [29]:
## reomve Tokens with the length less than 3
stemmed_token = [i for i in stemmed_token if len(i) >= 3]
len(stemmed_token)

5787

Change the connector between two words in collocation to double underline

In [30]:
store_token = []
for word in stemmed_token: ## loop all word in list
    extracted_token = [] ##store word for the moment
    for i in range(len(word)):
        extracted_token.append(word[i].replace('_','__')) ## if a word contain '_', replace it to '__'
    extracted_token = "".join(extracted_token)
    store_token.append(extracted_token)
store_token.sort()
store_token

['ADMM',
 'ADMM',
 'ADMM',
 'ADMM',
 'ANR',
 'ANR',
 'ANR',
 'AUC',
 'AUC',
 'AUC',
 'Accuracy',
 'Acknowledgements',
 'Acknowledgments',
 'Ada__Newton',
 'Ada__Newton',
 'Adam',
 'Adaptive',
 'Additionally',
 'Advanced',
 'Adversarial',
 'After',
 'Agency',
 'Air',
 'Alg',
 'Algorithm',
 'Algorithms',
 'All',
 'Also',
 'Although',
 'Amazon',
 'Analysis',
 'And',
 'Another',
 'Any',
 'Appendix',
 'Application',
 'Applications',
 'Apply',
 'Approach',
 'Approximate',
 'Approximation',
 'Architecture',
 'Arora',
 'Assume',
 'Assumption',
 'Assumptions',
 'Average',
 'Avg',
 'Award',
 'BFGS',
 'BFGS',
 'BFGS',
 'BFGS',
 'Bach',
 'Background',
 'Based',
 'Basic',
 'Bayes',
 'Bayes__classifier',
 'Bayesian',
 'Beach',
 'Because',
 'Before',
 'Bellman',
 'Bellman__error',
 'Bengio',
 'Bernoulli',
 'Best',
 'Boltzmann',
 'Both',
 'Bottom',
 'Bound',
 'Bounds',
 'Brain',
 'Brownian__motion',
 'CAREER',
 'CAREER',
 'CAREER',
 'CAREER',
 'CAREER',
 'CAREER',
 'CDF',
 'CDF',
 'CDF',
 'CIFAR',
 'C

In [32]:
##remove all stop_word again
store_token = [w for w in store_token if w not in stopwords_set]## append words from tokens who is not stop word
len(store_token)

5513

### 4.3 Output vocabulary_index file and count_vector file

Now we get the clean token. So for each token we need to ensure it has index.

In [33]:
token_set = list(set(store_token))## remove repeting words
token_set.sort()
vocab_index = {}
for i in range(len(token_set)):##token_string:token_index format
    vocab_index[i] = token_set[i]
vocab_index

{0: 'ADMM',
 1: 'ANR',
 2: 'AUC',
 3: 'Accuracy',
 4: 'Acknowledgements',
 5: 'Acknowledgments',
 6: 'Ada__Newton',
 7: 'Adam',
 8: 'Adaptive',
 9: 'Additionally',
 10: 'Advanced',
 11: 'Adversarial',
 12: 'After',
 13: 'Agency',
 14: 'Air',
 15: 'Alg',
 16: 'Algorithm',
 17: 'Algorithms',
 18: 'All',
 19: 'Also',
 20: 'Although',
 21: 'Amazon',
 22: 'Analysis',
 23: 'And',
 24: 'Another',
 25: 'Any',
 26: 'Appendix',
 27: 'Application',
 28: 'Applications',
 29: 'Apply',
 30: 'Approach',
 31: 'Approximate',
 32: 'Approximation',
 33: 'Architecture',
 34: 'Arora',
 35: 'Assume',
 36: 'Assumption',
 37: 'Assumptions',
 38: 'Average',
 39: 'Avg',
 40: 'Award',
 41: 'BFGS',
 42: 'Bach',
 43: 'Background',
 44: 'Based',
 45: 'Basic',
 46: 'Bayes',
 47: 'Bayes__classifier',
 48: 'Bayesian',
 49: 'Beach',
 50: 'Because',
 51: 'Before',
 52: 'Bellman',
 53: 'Bellman__error',
 54: 'Bengio',
 55: 'Bernoulli',
 56: 'Best',
 57: 'Boltzmann',
 58: 'Both',
 59: 'Bottom',
 60: 'Bound',
 61: 'Bounds'

Moreover, The vocabulary_index file can be opened and edited to store `vocabulary:index`

In [34]:
## vocabulary and inex
with open('Group023_vocab.txt','w+',encoding='UTF-8') as vocafile:
    for i in vocab_index.keys():
        vocafile.write(vocab_index[i] + ':' + str(i) + '\n')

Now we need to get the word count for each token, and each token should match one unique index. `Counter function` is easy to use to get the word count. Finally wirte them into file.

In [35]:
def vocab_to_index(pid):
    token = tokenizebody(pid)[1]##get the token
    ## replace word to index
    for index in range(len(token)):
        for i in vocab_index.keys():
            if token[index] == vocab_index[i]:
                token[index] = i
    ##filter the tokens who is satisfactory
    token = [str(x) for x in token]
    token = [num for num in token if num.isdigit() == True]##because just matched token has transformed to number
    token = [int(x) for x in token]
    ##word count
    word_fre = Counter(token)
    
    return word_fre
vocab_to_index('PP7260')

Counter({2779: 15,
         2791: 1,
         2085: 2,
         2441: 1,
         3070: 1,
         2736: 1,
         2086: 22,
         824: 26,
         2479: 1,
         870: 4,
         1379: 3,
         2346: 2,
         1257: 4,
         1609: 4,
         2143: 16,
         1158: 5,
         2508: 9,
         2253: 1,
         1919: 1,
         676: 6,
         1081: 1,
         1016: 1,
         2458: 2,
         1097: 27,
         2810: 5,
         1378: 1,
         1971: 2,
         2329: 14,
         2278: 8,
         95: 1,
         345: 1,
         240: 1,
         397: 1,
         503: 1,
         332: 1,
         286: 1,
         49: 1,
         533: 1,
         1086: 1,
         2045: 2,
         2084: 9,
         1849: 4,
         238: 1,
         86: 1,
         22: 3,
         233: 1,
         1182: 1,
         3015: 2,
         1870: 2,
         1432: 4,
         1150: 1,
         2503: 1,
         2769: 2,
         2700: 13,
         1721: 3,
         1955: 1,
     

In [36]:
## Write count_vectors file
with open('Group023_count_vectors.txt','w+',encoding='UTF-8') as outputfile:
    for paper_id in all_pid:
        count_vectors = str(paper_id) + ',' + str(vocab_to_index(paper_id))[9:-2] +'\n'##[9:-2]to get vector and its frequency
        count_vectors = count_vectors.replace(' ','')
        outputfile.write(count_vectors)


## 5. Statistics Generation

## 5.1 Match Data

After opening the original file, we altered all special markers to space bars. Specially, in the original file, finding that some special characters cannot match ‘regular expression' package, so we changed these special characters to normal characters. Then, we could locate the needed files from the beginning of `abstract` to the end of `Paper Body` in the file and utilize regular expression to group .

In [37]:
def get_abs(url):
    txt_name = get_txt_name(url)
    with open(txt_name,'r',encoding='UTF-8') as content:
        text = content.read()
    
    text = text.replace('\n',' ')
    text = text.replace('?','')
    text = text.replace('¿','')
    text = text.replace('ﬃ','ffi')
    text = text.replace('ﬂ','fl')
    text = text.replace('ﬀ','ff')
    text = text.replace('ﬁ','fi')
    
    abs_pattern = 'Abstract(.*)1\sPaper\sBody'
    abstract = re.search(abs_pattern,text)
    abstract = abstract.group(1)
    return abstract

get_abs(inputfile[0])

'  We present the first truly polynomial algorithm for learning the struc- ture of bounded-treewidth junction trees – an attractive subclass of prob- abilistic graphical models that permits both the compact representation of probability distributions and efficient exact inference. For a constant treewidth, our algorithm has polynomial time and sample complexity, and provides strong theoretical guarantees in terms of $KL$ divergence from the true distribution. We also present a lazy extension of our approach that leads to very significant speed ups in practice, and demonstrate the viability of our method empirically, on several real world datasets. One of our key new theoretical insights is a method for bounding the con- ditional mutual information of arbitrarily large sets of random variables with only a polynomial number of mutual information computations on fixed-size subsets of variables, when the underlying distribution can be approximated by a bounded treewidth junction tree.  '

Locate the needed files from the beginning to the end of `Authored by` in the file and utilize regular expression to group .

In [38]:
def get_title(url):
    txt_name = get_txt_name(url)
    with open(txt_name,'r',encoding='UTF-8') as content:
        text = content.read()
    
    text = text.replace('\n',' ')
    text = text.replace('?','')
    text = text.replace('¿','')
    text = text.replace('ﬃ','ffi')
    text = text.replace('ﬂ','fl')
    text = text.replace('ﬀ','ff')
    text = text.replace('ﬁ','fi')

    title_pattern = '(.*)\sAuthored by'
    title = re.search(title_pattern,text)
    title = title.group(1)
    return title
get_title(inputfile[0])

'Efficient Principled Learning of Thin Junction  Trees '

Locate the needed files from the beginning of `Authored by` to the end of `Abstract` in the file and utilize regular expression to group .

In [39]:
def get_author(url):
    txt_name = get_txt_name(url)
    with open(txt_name,'r',encoding='UTF-8') as content:
        text = content.read()
    
    text = text.replace('\n',',')
    text = text.replace('?','')
    text = text.replace('¿','')
    text = text.replace('ﬃ','ffi')
    text = text.replace('ﬂ','fl')
    text = text.replace('ﬀ','ff')
    text = text.replace('ﬁ','fi')
    
    author_pattern = 'Authored by:,(.*),Abstract'
    author = re.search(author_pattern,text)
    
    author = author.group(1)
    author = author.strip(',')
    
    author = author.split(',')
    while '' in author:
        author.remove('')
    return author
get_author(inputfile[5])

['Manfred Opper',
 'Dan Cornford',
 'John S. Shawe-taylor',
 'Cdric Archambeau',
 'Yuan Shen']

## 5.2 Filter Data

Then, I counted abstracts from extracts. Due to the statistics needs, I had to transcribe the content of article into lower case. Firstly, I put the matched files into the training model, the file could be recognized whether these are a sentence or not through the training model. And then, these matched files would be transcribed to lower case, and I used `regular expression` to match the article which had been trained. Because of the question asking us to delete the sentence less 3 words, I adopted `stop_word` to match `regular expression` from the deleted sentences less than three words. 

In [40]:
def abs_stats(url):
    abstract = get_abs(url)
    
    ##lower casing the first charater of a sentence(abstract)
    tokenizer = PunktSentenceTokenizer(trainer.get_params())
    sentences = tokenizer.tokenize(abstract)
    sentences_lower = [s[0].lower()+s[1:] for s in sentences]
    abstract = "".join(sentences_lower)
    
    ##unigram
    tokenizer = RegexpTokenizer(r"[A-Za-z]\w+(?:[-'?]\w+)?")
    abs_tokens = tokenizer.tokenize(abstract)
    
    ##remove tokens with the length less than 3
    stopped_abs_tokens = [w for w in abs_tokens if w not in stopwords_set]
    
    return stopped_abs_tokens

abs_stats(inputfile[0])[0:10]

['We',
 'present',
 'polynomial',
 'algorithm',
 'learning',
 'struc',
 'ture',
 'bounded-treewidth',
 'junction',
 'trees']

Put the title's data which used `regular expression` into the training model., Then。we adopted `stop_word` to match `regular expression` from the deleted sentences less than three words. 

In [41]:
def title_stats(url):
    
    title = get_title(url)
    ##lower case title
    title = title.lower()
    
    ##unigram
    tokenizer = RegexpTokenizer(r"[A-Za-z]\w+(?:[-'?]\w+)?")
    title_token = tokenizer.tokenize(title)
    
    ##remove tokens with the length less than 3
    stopped_title_tokens = [w for w in title_token if w not in stopwords_set]
    return stopped_title_tokens

title_stats(inputfile[0])

['efficient', 'principled', 'learning', 'thin', 'junction', 'trees']

## 5.3 Statistic Data

Finally, we ranked the data which had been analyzed. According to the ask from the question, we would rank the top ten of `Abstract`, `Title` and `Author`. First of all, we built a list, put the analyzed data into the loop and used `Collections package` to count the frequency of characters showing up in the data. Then, after ranking it, we recorded the Abstract of top ten into the list.

In [42]:
## top 10 terms in abstract
abstract = []
for i in range(len(inputfile)):
    abstract.extend(abs_stats(inputfile[i]))
abstract_count = collections.Counter(abstract)
abstract_count = sorted(abstract_count.items(), key=lambda d: d[1], reverse=True)

top_abstract = []
for i in abstract_count[0:10]:
    top_abstract.append(i[0])
top_abstract

['learning',
 'data',
 'model',
 'algorithm',
 'show',
 'method',
 'problem',
 'models',
 'algorithms',
 'based']

Used `Collections package` to count the frequency of characters showing up in the data. Then, after ranking it, we recorded the Titles of top ten into the list.

In [43]:
## top 10 terms in title
title = []
for i in range(len(inputfile)):
    title.extend(title_stats(inputfile[i]))
title_count = collections.Counter(title)
title_count = sorted(title_count.items(), key=lambda d: d[1], reverse=True)
title_count[0:11]

top_title = []
for i in title_count[0:10]:
    top_title.append(i[0])
top_title

['learning',
 'models',
 'inference',
 'regression',
 'networks',
 'deep',
 'stochastic',
 'neural',
 'variational',
 'analysis']

Used `Collections package` to count the frequency of characters showing up in the data. Then, after ranking it, we recorded the Authors of top ten into the list.

In [44]:
## top 10 authors
author = []
for i in range(len(inputfile)):
    author.extend(get_author(inputfile[i]))
author_count = collections.Counter(author)
author_count = sorted(author_count.items(), key=lambda d: d[1], reverse=True)

top_author = []
for i in author_count[0:10]:
    top_author.append(i[0])
top_author

['Jan R. Peters',
 'Rong Jin',
 'Tianbao Yang',
 'Franois Fleuret',
 'Elad Hazan',
 'Manfred Opper',
 'Eric P. Xing',
 'Gerhard Neumann',
 'Shie Mannor',
 'Huan Xu']

In [45]:
## create stats dataframe and output to csv file
stats = pd.DataFrame([top_abstract,top_title,top_author])
stats = stats.T
stats.columns = ['top10_terms_in_abstracts', 'top10_terms_in_titles', 'top10_authors']
stats.to_csv("Group023_stats.csv",index = False)
stats

,top10_terms_in_abstracts,top10_terms_in_titles,top10_authors
0,learning,learning,Jan R. Peters
1,data,models,Rong Jin
2,model,inference,Tianbao Yang
3,algorithm,regression,Franois Fleuret
4,show,networks,Elad Hazan
5,method,deep,Manfred Opper
6,problem,stochastic,Eric P. Xing
7,models,neural,Gerhard Neumann
8,algorithms,variational,Shie Mannor
9,based,analysis,Huan Xu


## 6. Summary

This assessment measured the understanding of basic text file processing techniques in the Python programming language and extracted data into a proper format suitable for a downstream modelling task.
- **Extraction** Defining function is a clear and user friendly way to parse these patent features. [regex101](https://regex101.com/) is a very useful online tool to check the regular expression. And also it can show which part is the group of regular expression. The `re.search` and `re.sub` is used frequently in this part to extract the particular text from main file. Then by using basic funtion like `if else` and `loop` to make judgment and appending function.


- **Sparse Feature Generation** Firstly, using `tokenization` function to identify the file sentence segmentation. Then, define a `RegexTokenizer` to tokenize all words from paper. Secondly, due to removed vocabularys, put the data into the stop words. Check the difference between th output of `MWEtokenizer` and `RegexpTokenizer` by adpating the following code to make sure we get the right collocation. Rare tokens (with the threshold set to 3%) and meaningless tokens (with the threshold set to 95%) must be removed from the vocab. Thirdly, Using `Porter stemmer` to stem Unigram tokens. Finaly, output vocabulary_index file and count_vector file respectively.


- **Statistics Generation** First of all, altering all special markers to space bars. Specially, in the original file, finding that some special characters cannot match `regular expression` package, so we changed these special characters to normal characters. According assignment requirement, the word tokenization must use the following regular expression, `r"[A-Za-z]\w+(?:[-'?]\w+)?"`. Then, put the matched files into the training model, the file could be recognized whether these are a sentence or not through the training model. Finally,  put the analyzed data into the loop and used `Collections package` to count the frequency of characters showing up in the data.

# 7. References

- The `pandas` Project. (2016a). *pandas 0.19.2 documentation: pandas.DataFrame.to_csv*. Retrieved from http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.to_csv.html#pandas.DataFrame.to_csv
- tokenize — Tokenizer for Python source*. Retrieved from https://docs.python.org/3/library/tokenize.html#module-tokenize
- Python | Stemming words with NLTK*. Retrieved from https://www.geeksforgeeks.org/python-stemming-words-with-nltk/
